In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [42]:
#Data Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081))
])

In [43]:
#loading mnist dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform
)

In [44]:
#dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [45]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784,128),
            nn.ReLU(),
            nn.Linear(128,10)
        )
    def forward(self, x):
        x = self.flatten(x)
        x = self.layers(x)
        return x

In [46]:
#check gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}')

Using cpu


In [47]:
#intialize model
model = MNISTClassifier() 

In [48]:
#loss function, optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [53]:
#training
def train_epoch(model, train_loader, loss_function, optimizer, device):
    model.train() #puts model in training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        #track progress
        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        #print every 100 batches
        if batch_idx % 100 == 0 and batch_idx > 0:
            avg_loss = running_loss / 100
            accuracy = 100. * correct / total
            print(f' [{batch_idx * 64}/{60000}] '
                  f' Loss: {avg_loss:.3f} | Accuracy: {accuracy:.1f}%')
            running_loss = 0.0

In [54]:
#evaluation
def evaluate(model, test_loader, device):
    model.eval() #set to evaluation mode 
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return 100. * correct / total

In [55]:
#training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f'\nEpoch: {epoch+1}')
    train_epoch(model, train_loader, loss_function, optimizer, device)
    accuracy = evaluate(model, test_loader, device)
    print(f'Test Accuracy: {accuracy:.2f}%')


Epoch: 1
 [6400/60000]  Loss: 0.123 | Accuracy: 96.4%
 [12800/60000]  Loss: 0.119 | Accuracy: 96.4%
 [19200/60000]  Loss: 0.124 | Accuracy: 96.4%
 [25600/60000]  Loss: 0.107 | Accuracy: 96.5%
 [32000/60000]  Loss: 0.114 | Accuracy: 96.5%
 [38400/60000]  Loss: 0.105 | Accuracy: 96.6%
 [44800/60000]  Loss: 0.102 | Accuracy: 96.6%
 [51200/60000]  Loss: 0.088 | Accuracy: 96.7%
 [57600/60000]  Loss: 0.108 | Accuracy: 96.7%
Test Accuracy: 96.55%

Epoch: 2
 [6400/60000]  Loss: 0.077 | Accuracy: 97.6%
 [12800/60000]  Loss: 0.073 | Accuracy: 97.8%
 [19200/60000]  Loss: 0.085 | Accuracy: 97.7%
 [25600/60000]  Loss: 0.077 | Accuracy: 97.7%
 [32000/60000]  Loss: 0.072 | Accuracy: 97.7%
 [38400/60000]  Loss: 0.071 | Accuracy: 97.7%
 [44800/60000]  Loss: 0.079 | Accuracy: 97.7%
 [51200/60000]  Loss: 0.072 | Accuracy: 97.6%
 [57600/60000]  Loss: 0.088 | Accuracy: 97.6%
Test Accuracy: 96.89%

Epoch: 3
 [6400/60000]  Loss: 0.052 | Accuracy: 98.5%
 [12800/60000]  Loss: 0.060 | Accuracy: 98.3%
 [19200/6